<a href="https://www.kaggle.com/code/msbluphire/sme-ghost-protocol?scriptVersionId=253020934" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# When Your SME Ghosts You: A GenAI-Powered Solution for Database Documentation



# -------------------------
# 1. Project Overview
# -------------------------
# 🧠 The Database Documentation Agent is a GenAI-powered assistant designed to help analysts, product managers, and data professionals quickly understand and interact with relational databases — especially unfamiliar or undocumented ones.

 👤 Primary User Persona: Alexis, a business systems analyst, frequently receives SQLite data dumps from vendors and internal teams with little to no documentation. She needs to understand schemas, relationships, and data content quickly in order to validate business logic, design reports, or draft integration specs.

# 🔍 MVP Capabilities:
 - Automatically extracts and summarizes table schemas using Gemini
 - Allows users to ask natural language questions about the schema (e.g., relationships, column meanings)
 - Generates AI-powered SQL queries based on business-style questions and explains them in plain English
 - Visualizes the schema as an ER diagram with primary and foreign key connections
 - Provides a mockup of a web UI for future deployment via Streamlit or Flask

 # 💡 Why It Matters:
 Traditional database tools focus on technical metadata. This agent bridges the gap between structured data and business understanding, allowing users to get answers without deep SQL knowledge — all from a local SQLite file.

 ✅ Fully self-contained — no need for external database setup, making it ideal for rapid prototyping, data audits, or one-off investigations.


# -------------------------
# 1.1 Project Development Workflow
# -------------------------
 This capstone was developed using a structured product development lifecycle to reflect how real-world AI tools are designed and delivered. Here's how the project was built:

# 📌 Step 1: Use Case Definition
 - Identified a common pain point for business analysts working with unfamiliar relational databases.
 - Defined Alexis (analyst persona) as the primary user.
 - Captured user needs: schema clarity, query assistance, documentation generation, and visual comprehension.

# 📝 Step 2: Product Requirements Document (PRD)
  - Outlined core MVP features:
   - Schema extraction
   - Table summarization
   - AI-powered Q&A
   - SQL query generation
   - Schema diagramming
   - Included non-functional goals like local/offline support, clean UI (future), and Gemini-powered intelligence.

# 📅 Step 3: Agile Sprint Planning
 - Broke features into sprints:
   - Sprint 1: Database setup + schema parsing
   - Sprint 2: Table summaries with Gemini
   - Sprint 3: Q&A over schema using natural language
   - Sprint 4: SQL generation and result explanation
   - Sprint 5: Schema visualizer (Graphviz + inline display)
   - Sprint 6: Polish, error handling, future roadmap

# 🔁 Step 4: Iterative Development & Testing
 - Built in a modular way using test inputs and printed outputs
 - Handled edge cases (e.g., query errors, schema loading)
 - Collected feedback on prompts and outputs to improve quality

# 🚀 Outcome: A working prototype that translates structured database metadata into human-friendly insights using GenAI.


# 2. Setup - Gemini API Key Configuration

This section configures the connection to Gemini (Google's Generative AI). Make sure your Kaggle secrets are properly set up.


In [ ]:
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

# Access the API key from Kaggle Secrets
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("GOOGLE_API_KEY")

# Configure the Gemini API with the retrieved API key
genai.configure(api_key=api_key)  # Correct configuration

# Select the appropriate model
model = genai.GenerativeModel('gemini-2.5-flash-lite')

print("✅ Gemini API configured successfully with the model: gemini-2.5-flash-lite")


# 3. Database Setup (SQLite + pandas)

This section creates an in-memory SQLite3 database with tables for `Customers`, `Products`, `Orders`, and `OrderItems` to simulate a business system.


In [ ]:
import sqlite3
import pandas as pd

# Create in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Define schema and sample data
cursor.execute('''CREATE TABLE Customers (
    Customer_ID INTEGER PRIMARY KEY,
    First_Name TEXT,
    Last_Name TEXT,
    Email TEXT,
    Phone_Number TEXT,
    Address TEXT
)''')

cursor.execute('''CREATE TABLE Products (
    Product_ID INTEGER PRIMARY KEY,
    Product_Name TEXT,
    Description TEXT,
    Price REAL
)''')

cursor.execute('''CREATE TABLE Orders (
    Order_ID INTEGER PRIMARY KEY,
    Customer_ID INTEGER,
    Order_Date TEXT,
    Order_Total REAL,
    Payment_Method TEXT,
    Shipping_Method TEXT,
    FOREIGN KEY (Customer_ID) REFERENCES Customers(Customer_ID)
)''')

cursor.execute('''CREATE TABLE OrderItems (
    OrderItem_ID INTEGER PRIMARY KEY,
    Order_ID INTEGER,
    Product_ID INTEGER,
    Quantity INTEGER,
    Price_Per_Item REAL,
    FOREIGN KEY (Order_ID) REFERENCES Orders(Order_ID),
    FOREIGN KEY (Product_ID) REFERENCES Products(Product_ID)
)''')

# Insert sample data into Customers
customers_data = [
    (1, 'Alex', 'Johnson', 'alex.j@example.com', '1234567890', '123 Elm Street'),
    (2, 'Ben', 'Smith', 'ben.s@example.com', '0987654321', '456 Oak Avenue')
]
cursor.executemany('INSERT INTO Customers VALUES (?, ?, ?, ?, ?, ?)', customers_data)

# Insert sample data into Products
products_data = [
    (1, 'Widget A', 'Standard widget', 9.99),
    (2, 'Widget B', 'Deluxe widget', 14.99)
]
cursor.executemany('INSERT INTO Products VALUES (?, ?, ?, ?)', products_data)

# Insert sample Orders
orders_data = [
    (1, 1, '2025-04-10', 29.97, 'Credit Card', 'UPS'),
    (2, 2, '2025-04-12', 19.98, 'PayPal', 'FedEx')
]
cursor.executemany('INSERT INTO Orders VALUES (?, ?, ?, ?, ?, ?)', orders_data)

# Insert sample OrderItems
order_items_data = [
    (1, 1, 1, 2, 9.99),
    (2, 1, 2, 1, 14.99),
    (3, 2, 1, 2, 9.99)
]
cursor.executemany('INSERT INTO OrderItems VALUES (?, ?, ?, ?, ?)', order_items_data)

conn.commit()

print("Database schema initialized. Tables created: Customers, Products, Orders, OrderItems")


# 4. Schema Extraction for GenAI

This section extracts the schema from the SQLite database and prepares the data in JSON format to use with Gemini.


In [ ]:
# Extract table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [row[0] for row in cursor.fetchall()]

schema_info = {}

# Extract schema for each table
for table in tables:
    cursor.execute(f"PRAGMA table_info({table});")
    columns = cursor.fetchall()
    schema_info[table] = [
        {
            "column_id": col[0],
            "name": col[1],
            "type": col[2],
            "notnull": bool(col[3]),
            "default_value": col[4],
            "is_primary_key": bool(col[5])
        }
        for col in columns
    ]

import json
print("\n\nStructured Schema JSON:")
print(json.dumps(schema_info, indent=2))


# 5. Table Summary Generation (Few-shot Prompting)

In this section, we'll generate a summary of each table in the database using Gemini's few-shot prompting capability.


In [ ]:
from typing import List

# Use Gemini to generate summaries for each table
model = genai.GenerativeModel(model_name="gemini-2.5-flash-lite")

def gemini_table_summary(table_name: str, columns: List[dict]) -> str:
    column_descriptions = ", ".join([f"{col['name']} ({col['type']})" for col in columns])
    prompt = f"""
    Summarize the purpose and contents of a SQL table based on the following details:

    Table name: {table_name}
    Columns: {column_descriptions}

    Provide a 2-3 sentence description appropriate for business analysts and product managers.
    """
    try:
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        return f"[Error generating summary for {table_name}]: {e}"

print("\n\nTable Summaries (Generated by Gemini):")
for table, cols in schema_info.items():
    print(f"\n■ {table}:")
    print(gemini_table_summary(table, cols))


# 6. Q&A Over Schema (Natural Language Queries)

This section demonstrates how you can use Gemini to answer natural language questions about the schema.


In [ ]:
def gemini_schema_qa(question: str, schema: dict) -> str:
    formatted_schema = json.dumps(schema, indent=2)
    prompt = f"""
    You are a helpful assistant. A user has asked a question about a database schema. 
    Use the schema below to answer as accurately and clearly as possible.

    ---
    Schema:
    {formatted_schema}

    ---
    Question: {question}
    Answer:
    """
    try:
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        return f"[Error answering question]: {e}"

# Example usage:
print("\n\nQ&A Examples:")
questions = [
    "Which tables reference the Products table?",
    "What columns are used to link Orders and Customers?",
    "How can I find out which products a customer ordered?"
]
for q in questions:
    print(f"\n❓ {q}")
    print(gemini_schema_qa(q, schema_info))


# 7. Interactive Q&A (Gemini-Powered)

This section enables interactive Q&A for users to ask questions about the database schema in real time. Type 'exit' to quit the loop.


In [ ]:
print("\n🔁 Enter interactive Q&A mode (type 'exit' to quit):")
while True:
    try:
        user_input = input("\nAsk a schema question: ")
        if user_input.lower() in ['exit', 'quit']:
            print("👋 Exiting Q&A loop.")
            break
        response = gemini_schema_qa(user_input, schema_info)
        print(f"\n💬 Gemini says: {response}")
    except Exception as e:
        print(f"⚠️ Error during Q&A loop: {e}")



# 8. Advanced Query Insights (AI-Powered SQL)

In this section, we enable the Database Documentation Agent to translate **natural language business questions** into **SQLite-compatible SQL queries**, execute them on the live database, and provide **explanations in plain English**.

This helps non-technical users extract meaningful insights from the data without writing a single line of SQL.

✨ Features included:
- Gemini generates SQL queries using schema context.
- Queries are executed with results returned via `pandas`.
- Gemini explains what each query does in user-friendly language.

🔁 You can test this capability interactively by entering business questions in the loop below.


In [ ]:
# -------------------------
# 8. Advanced Query Insights (AI-Powered SQL)
# -------------------------
import traceback

def gemini_sql_generator(natural_question: str, schema: dict) -> str:
    formatted_schema = json.dumps(schema, indent=2)
    prompt = f"""
    You are an expert SQL assistant. Based on the database schema below, write an SQLite-compatible SQL query to answer the user's question.

    Use only table and column names that exist. Return the query only — do not explain it. Do not wrap the query in markdown formatting.

    ---
    Schema:
    {formatted_schema}

    ---
    User Question: {natural_question}
    SQL Query:
    """
    try:
        response = model.generate_content(prompt)
        raw_sql = response.text.strip()
        # Clean up any markdown code fencing
        cleaned_sql = raw_sql.strip("`").replace("```sql", "").replace("```sqlite", "").replace("```", "").strip()
        return cleaned_sql
    except Exception as e:
        return f"[Error generating SQL]: {e}"

def run_advanced_query_loop():
    print("\n🧠 Advanced Query Mode (type 'exit' to quit):")
    while True:
        try:
            user_q = input("\nAsk a business question: ")
            if user_q.lower() in ['exit', 'quit']:
                print("👋 Exiting advanced query loop.")
                break

            generated_sql = gemini_sql_generator(user_q, schema_info)
            print(f"\n📝 Generated SQL:\n{generated_sql}")

            if generated_sql.lower().startswith("[error"):
                print(f"⚠️ {generated_sql}")
                continue

            df = pd.read_sql_query(generated_sql, conn)
            print(f"\n📊 Query Result:")
            print(df)

            explain_prompt = f"Explain the following SQL query in simple terms:\n\n{generated_sql}"
            explanation = model.generate_content(explain_prompt).text.strip()
            print(f"\n💡 Explanation:\n{explanation}")

        except Exception as e:
            print("⚠️ Error during advanced query processing:")
            traceback.print_exc()

run_advanced_query_loop()



# 9. Schema Visualizer (ER Diagram Generator)


In this section, we use **Graphviz** to create an **Entity-Relationship (ER) diagram** based on the database schema. The goal is to visually represent the relationships between tables and columns, making it easier for users (like business analysts) to quickly understand the database structure.

We will extract the schema from the SQLite database and then use **Graphviz** to generate a visual representation of the tables and their relationships. 

The output will include:
- **Nodes** representing tables.
- **Edges** representing relationships between tables (foreign keys).

Let's generate the schema diagram based on the existing tables in our SQLite database.


In [ ]:
# -------------------------
# 9. Schema Visualizer (Markdown + Graphviz)
# -------------------------
# This section visualizes the database schema as an entity-relationship diagram using Graphviz.
# Tables are shown as boxes, columns as ellipses, with primary keys in blue and foreign keys linked in red.

import graphviz

def generate_schema_graph(schema_info, conn):
    dot = graphviz.Digraph(format='png', engine='dot')

    # Add table nodes
    for table_name, columns in schema_info.items():
        dot.node(table_name, label=table_name, shape='box')
        for column in columns:
            col_node = f"{table_name}_{column['name']}"
            label = f"{column['name']}"
            if column['is_primary_key']:
                label += " (PK)"
                dot.node(col_node, label=label, shape='ellipse', color='blue')
            else:
                dot.node(col_node, label=label, shape='ellipse')
            dot.edge(table_name, col_node)

    # Foreign key relationships
    cursor = conn.cursor()
    for table_name in schema_info.keys():
        cursor.execute(f"PRAGMA foreign_key_list({table_name});")
        for fk in cursor.fetchall():
            from_col = f"{table_name}_{fk[3]}"
            to_col = f"{fk[2]}_{fk[4]}"
            dot.edge(from_col, to_col, color='red')

    return dot

schema_graph = generate_schema_graph(schema_info, conn)
schema_graph.render('schema_diagram', format='png', cleanup=False)
print("✅ Schema diagram rendered to 'schema_diagram.png'")



In [ ]:
# -------------------------
# 9.1 View Schema Diagram Inline
# -------------------------
# This section renders the ER diagram directly inside the notebook using IPython's display module.

from IPython.display import Image, display

# Display the rendered schema diagram inside the notebook
display(Image(filename='schema_diagram.png'))



 # 10. Future Enhancements
 
 ✅ CLI Tool (Command Line Interface)
 - Provide a terminal-friendly experience for quick insights, schema summaries, and AI-powered SQL.
 - Support optional arguments like `--schema-summary`, `--qa`, `--query="..."`.

 ✅ Web UI (MVP Mockup in Streamlit or Flask)
 - Users select a local `.db` or `.sqlite` file.
 - App auto-parses schema and uses Gemini to generate documentation.
 - Includes:
     - Table viewer with schema highlights
     - Input box for natural language queries
     - SQL preview + Gemini explanation
     - Export option for generated docs (PDF, Markdown, etc.)

 ✅ Interactive Schema Graph (Future)
 - Replace Graphviz with Pyvis or Plotly for zoomable, clickable diagrams
 - Hover over columns to see details like type, keys, and relationships
 - Link nodes to AI-generated summaries or example queries

 ✅ Advanced GenAI Enhancements (Future Ideas)
 - **Schema & Metadata Embeddings**: Use vector embeddings for schema parts to enable semantic search and natural query-to-schema alignment.
 - **Memory-Powered Q&A**: Retrieve past interactions and summaries using RAG techniques for more accurate responses.
 - **Conversational Agent**: Support follow-up questions and maintain conversation context.
 - **Explainability Engine**: Auto-describe relationships, column use cases, and generated SQL.
 - **Doc Export Pipeline**: Format results as markdown, PDF, or integrate with tools like Notion or Confluence.